# Preprocessing

In [23]:
import json
import uuid

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from difflib import SequenceMatcher
from IPython.display import HTML

pd.set_option('display.max_columns', None)

codenet_root = 'datasets/codenetpy/'

ModuleNotFoundError: No module named 'editdistance'

In [2]:
with open(codenet_root + 'codenetpy_train.json', 'r') as f:
    data = json.load(f)["data"]
    
with open(codenet_root + 'codenetpy_test.json', 'r') as f:
    test_data = json.load(f)["data"]
    

train_df = pd.DataFrame(data)
test_df = pd.DataFrame(test_data)

codenetpy_df = pd.DataFrame(data + test_data)

display(codenetpy_df)
display(codenetpy_df.info())

,original_src,changed_src,problem_id,original_id,changed_id,language,filename_ext,original_status,returncode,error_class,error_class_extra,error,output
0,l=[]\nwhile True:\n try:\n l.append(...,l=[]\nwhile True:\n try:\n l.append(...,p00001,s196059089,s508355022,Python,py,Runtime Error,1,SyntaxError,SyntaxError: Missing parentheses in call to 'p...,"File ""/home/alex/Documents/research/bug-dete...",
1,"data = []\nfor i in range(0,10):\n data.app...","data = []\nfor i in range(0,10):\n data.app...",p00001,s840313913,s627573591,Python,py,WA: Presentation Error,0,0,,,\n3776\n2848\n2840\n
2,first = 0;\nsecond = 0;\nthird = 0;\nfor var i...,first = 0;\nsecond = 0;\nthird = 0;\nfor var i...,p00001,s973623080,s521689663,Python,py,Runtime Error,1,TypeError,TypeError: '>' not supported between instances...,"Traceback (most recent call last):\n File ""/h...",
3,for i in range(10):\n s.append(int(input())...,s=[]\nfor i in range(10):\n s.append(int(in...,p00001,s393051088,s459982723,Python,py,Runtime Error,1,NameError,NameError: name 's' is not defined,"Traceback (most recent call last):\n File ""/h...",
4,# -*- coding: utf-8 -*-\n\nimport sys\n\ndef t...,# -*- coding: utf-8 -*-\n\nimport sys\n\ndef t...,p00001,s803831828,s403126268,Python,py,Runtime Error,1,NameError,NameError: name 'argv' is not defined,"Traceback (most recent call last):\n File ""/h...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54587,"N = int(input())\nL = list(map(int, input().sp...","N = int(input())\nL = list(map(int, input().sp...",p04047,s420396881,s036877571,Python,py,Runtime Error,1,IndexError,IndexError: list index out of range,"Traceback (most recent call last):\n File ""/h...",
54588,\nnum = int(input())\nli = int(input().split()...,n = int(input())\nli = [int(x) for x in input(...,p04047,s838662080,s028090472,Python,py,Runtime Error,1,TypeError,"TypeError: int() argument must be a string, a ...","Traceback (most recent call last):\n File ""/h...",
54589,from collections import Counter\n \nn = int(in...,"n = int(input())\n \narr = list(map(int, input...",p04047,s112597968,s888016571,Python,py,Runtime Error,1,TypeError,TypeError: unsupported operand type(s) for +=:...,"Traceback (most recent call last):\n File ""/h...",
54590,"N,X = input().split()\nN,X = int(N), int(X)\na...","N,X = input().split()\nN,X = int(N), int(X)\na...",p04048,s311100241,s323769427,Python,py,Runtime Error,1,TabError,TabError: inconsistent use of tabs and spaces ...,"File ""/home/alex/Documents/research/bug-dete...",


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54592 entries, 0 to 54591
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   original_src       54592 non-null  object
 1   changed_src        54592 non-null  object
 2   problem_id         54592 non-null  object
 3   original_id        54592 non-null  object
 4   changed_id         54592 non-null  object
 5   language           54592 non-null  object
 6   filename_ext       54592 non-null  object
 7   original_status    54592 non-null  object
 8   returncode         54592 non-null  int64 
 9   error_class        54592 non-null  object
 10  error_class_extra  54592 non-null  object
 11  error              54592 non-null  object
 12  output             54592 non-null  object
dtypes: int64(1), object(12)
memory usage: 5.4+ MB


None

In [3]:
codenetpy_df['error_class'].value_counts()

0                            28406
SyntaxError                   8686
NameError                     5262
TypeError                     4202
ValueError                    2444
IndentationError              1081
IndexError                     999
AttributeError                 913
EOFError                       852
TLEError                       525
ModuleNotFoundError            357
TabError                       265
ImportError                    132
ZeroDivisionError               79
KeyError                        66
SyntaxWarning                   62
FileNotFoundError               62
UnboundLocalError               61
1                               58
DeprecationWarning              35
RecursionError                  13
OverflowError                   12
RuntimeError                     6
-11                              5
2                                3
OSError                          3
SparseEfficiencyWarning          1
255                              1
VisibleDeprecationWa

In [4]:
codenetpy_df['returncode'].value_counts()

 0      28455
 1      25603
-9        525
-11         5
 2          3
 255        1
Name: returncode, dtype: int64

In [5]:
# There are cases where the return code is 0 but the error_class is a Warning

codenetpy_df[(codenetpy_df['returncode'] == 0) & (codenetpy_df['error_class'] != '0')]['error_class'].value_counts()

DeprecationWarning         30
SyntaxWarning              18
SparseEfficiencyWarning     1
Name: error_class, dtype: int64

In [6]:
# Most of the '0' error_class buggy submissions are TLE errors, which means they are inefficient implementations

codenetpy_df[codenetpy_df['error_class'] == '0']['original_status'].value_counts()

Time Limit Exceeded       17252
Runtime Error              9590
WA: Presentation Error     1485
Memory Limit Exceeded        73
Output Limit Exceeded         5
Judge Not Available           1
Name: original_status, dtype: int64

In [19]:
# Idealy the model will take as input the original_src code and will identify both the error_description and the location of the bug
# We can achieve this by comparing the original_src with the changed_src, which will give us the location of the bug (e.g. the modification)
# Next we will want to create a new model that will take as input the original_src, and the found bug and will learn to make the modifications
# on the original code such that it gets accepted

buggy_df = codenetpy_df[codenetpy_df['returncode'] != 0]
buggy_df = buggy_df.rename(columns={'original_src': 'code_string'})
buggy_df['uuid'] = [uuid.uuid4().hex for _ in range(len(buggy_df.index))]
buggy_df = buggy_df.set_index('uuid')
buggy_df = buggy_df[['code_string']]

In [20]:
buggy_df.to_json(r'example.json', orient='index', indent=4)